# Lingyi's Project

-----

## Section 1 - Retrieve and Organize Data

#### -import necessary packages-

In [2]:
import pandas as pd
import zipfile
import requests
from pandas import DataFrame
import numpy as np
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### -retrieve data-

In [3]:
zf = zipfile.ZipFile('ma.csv.zip') 
df = pd.read_csv(zf.open('ma.csv'))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### -organize data-

In [4]:
df1 = df.drop(['ID','NUMBER','UNIT','HASH','REGION'],axis=1)
df2 = df1.dropna()
df3 = df2.drop_duplicates(subset='POSTCODE').reset_index(drop=True)

In [5]:
middlesex = df3[df3['DISTRICT'].str.contains('MIDDLESEX')]
suffolk = df3[df3['DISTRICT'].str.contains('SUFFOLK')]
norfolk = df3[df3['DISTRICT'].str.contains('NORFOLK')]
merge = norfolk.append(middlesex, ignore_index=True)
df = merge.append(suffolk,ignore_index = True)
df.head(12)

,LON,LAT,STREET,CITY,DISTRICT,POSTCODE
0,-71.376402,41.986877,FALES ROAD,PLAINVILLE,NORFOLK,2762
1,-71.494588,42.017492,CALIFORNIA AVENUE,BELLINGHAM,NORFOLK,2019
2,-71.457583,42.017947,JENKS STREET,WRENTHAM,NORFOLK,2093
3,-71.288031,42.017653,CEDAR STREET,FOXBOROUGH,NORFOLK,2035
4,-71.442234,42.037219,VAIL DRIVE,FRANKLIN,NORFOLK,2038
5,-71.177637,42.057936,MANSFIELD STREET,SHARON,NORFOLK,2067
6,-71.137240,42.074167,CLEARY DRIVE,STOUGHTON,NORFOLK,2072
7,-71.288320,42.085192,LAFAYETTE LANE,NORFOLK,NORFOLK,2056
8,-71.280177,42.089123,COBBLE KNOLL DRIVE,WALPOLE,NORFOLK,2071
9,-71.279831,42.101279,YONKER PLACE,WALPOLE,NORFOLK,2081


------

## Section 2 - Visualize Map

#### -Use geopy library to get the latitude and longitude values of Boston-


In [6]:
address = 'Boston, Massachusetts'
geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


#### -Create a map of Middlesex, Suffolk and Norfolk with neighborhoods superimposed on top-


In [33]:
# create map of Boston neighborhoods using latitude and longitude values
mapb = folium.Map(location =[latitude, longitude],zoom_start =9)

# add markers to map
for lat, lng, district, street in zip(df['LAT'],df['LON'],df['DISTRICT'],df['STREET']):
    label = '{},{}'.format(street, district)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 3,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#31b5cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(mapb)
mapb

-----

## Section 3.1 - Explore Neighborhood

#### -Define foursquare credentials and version-

In [8]:
CLIENT_ID = 'CFU5JPRQFUHRQNUTSOE4LSCP2MGFSXOAY4QTTUGARQV1VI3W'
CLIENT_SECRET = 'R3BQC1SMFA1XHLDKPDJ5MWID0BDTH5TQXJVVLOHW2KVML1YO' 
VERSION = '20200506'

#### _Define a function to explore all the neighborhoods in Toronto using foursquare-

In [9]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']   
    return(nearby_venues)

#### -Run the above function on each neighborhood and create a new dataframe called boston_venues-

In [10]:
boston_venues = getNearbyVenues(names=df['STREET'],
                                   latitudes=df['LAT'],
                                   longitudes=df['LON']
                                  )

FALES ROAD
CALIFORNIA AVENUE
JENKS STREET
CEDAR STREET
VAIL DRIVE
MANSFIELD STREET
CLEARY DRIVE
LAFAYETTE LANE
COBBLE KNOLL DRIVE
YONKER PLACE
FRANCIS STREET
MYRTLE STREET
MISHAWAUM STREET
WASHINGTON STREET
THICKET STREET
KRISTEN ROAD
HIGH PLAIN STREET
ONONDAGA LANE
BREWSTER DRIVE
LIVOLI AVENUE
PARK AVENUE WEST
WINSLOW ROAD
PLEASANT STREET
COUNTY STREET
BLUE HILL RIVER ROAD
BATES LANE
BLUE HILL DRIVE
CHICKATAWBUT ROAD
THOMPSON ROAD
RESERVOIR ROAD
WILSON LANE
SEALUND ROAD
SOUTH LINCOLN STREET
ROSEMARY STREET
WASHINGTON STREET
NICHOLS STREET
RICHARDSON ROAD
LUNENBURG ROAD
SUMAC DRIVE
SOUTH STREET
LUMBER STREET
TOWNE LYNE ROAD
DONNA LEE LANE
GLEN STREET
GUILD ROAD
SPIERS ROAD
WORCESTER ROAD
WALLACE STREET
OAK STREET
DANGELO DRIVE
OAK KNOLL ROAD
OAKDALE AVENUE
RADCLIFF ROAD
WASHINGTON STREET
KINMONTH ROAD
WASHINGTON STREET
LOWELL AVENUE
KENILWORTH STREET
DAWSON DRIVE
GRANITE STREET
SOUTH STREET
MAPLE STREET
MEMORIAL DRIVE
CARDINAL MEDEIROS AVENUE
PUTNAM AVENUE
FOX ROAD
CONLON COURT
SCHOOL 

In [11]:
print(boston_venues.shape)
boston_venues.groupby('Neighborhood').count()

(3368, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A STREET,82,82,82,82,82,82
ABBOTSFORD STREET,9,9,9,9,9,9
ABERDEEN STREET,39,39,39,39,39,39
ACADIA STREET,10,10,10,10,10,10
ACORN STREET,39,39,39,39,39,39
...,...,...,...,...,...,...
WINSLOW ROAD,2,2,2,2,2,2
WOOD LANE,2,2,2,2,2,2
WOODLAND ROAD,2,2,2,2,2,2


#### -Amount of unique categories curated from all the returned venues-

In [12]:
print('There are {} uniques categories.'.format(len(boston_venues['Venue Category'].unique())))

There are 305 uniques categories.


------

## Section 3.2 Analyze each Neighborhood

In [13]:
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#new dataframe size
boston_onehot.shape

(3368, 305)

#### -Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category-

In [15]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Aquarium,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,A STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.012195,0.0,0.0,...,0.000000,0.012195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABBOTSFORD STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABERDEEN STREET,0.025641,0.0,0.0,0.0,0.0,0.0,0.051282,0.0,0.0,...,0.025641,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ACADIA STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ACORN STREET,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,WINSLOW ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,WOOD LANE,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,WOODLAND ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149,WORCESTER ROAD,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#new size
boston_grouped.shape

(151, 305)

#### -Top 5 most common venues of each neighborhood-

In [17]:
num_top_venues = 5

for hood in boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A STREET----
                venue  freq
0         Coffee Shop  0.07
1                 Bar  0.04
2               Hotel  0.04
3  Italian Restaurant  0.04
4  Seafood Restaurant  0.04


----ABBOTSFORD STREET----
                  venue  freq
0           Record Shop  0.11
1                 Plaza  0.11
2                Garden  0.11
3   Monument / Landmark  0.11
4  Fast Food Restaurant  0.11


----ABERDEEN STREET----
                 venue  freq
0               Bakery  0.05
1  American Restaurant  0.05
2         Burger Joint  0.05
3   Chinese Restaurant  0.05
4               Lounge  0.05


----ACADIA STREET----
                        venue  freq
0                        Park   0.2
1  Construction & Landscaping   0.1
2                       Diner   0.1
3              Breakfast Spot   0.1
4                        Café   0.1


----ACORN STREET----
                venue  freq
0           Gift Shop  0.05
1         Pizza Place  0.05
2  Italian Restaurant  0.05
3        Gourmet Shop  0.05
4   

#### -Put above datas into a Pandas dataframe & sort venues in descending order-

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### -create new dataframe and display top 10 venues in each neighborhood-

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A STREET,Coffee Shop,Hotel,Juice Bar,Italian Restaurant,Bar,Seafood Restaurant,Art Gallery,Clothing Store,New American Restaurant,Asian Restaurant
1,ABBOTSFORD STREET,Monument / Landmark,Plaza,Record Shop,Museum,American Restaurant,Pizza Place,Fast Food Restaurant,Pharmacy,Garden,Exhibit
2,ABERDEEN STREET,Lounge,American Restaurant,Chinese Restaurant,Bakery,Burger Joint,Yoga Studio,Mexican Restaurant,Beer Garden,Big Box Store,Furniture / Home Store
3,ACADIA STREET,Park,Pizza Place,Breakfast Spot,Café,Dive Bar,Diner,Construction & Landscaping,Ice Cream Shop,Convenience Store,Cupcake Shop
4,ACORN STREET,Gourmet Shop,French Restaurant,Pizza Place,Gift Shop,Italian Restaurant,Breakfast Spot,Theater,Kids Store,Bakery,Lake


-----

## Section 3.3 - Cluster Neighborhoods

#### -Run K-means to cluster the neighborhoods into 5 clusters-

In [20]:
# set number of clusters
kclusters = 5

boston_grouped_clustering = boston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

#### -Create new dataframe to include clusters as well as top 10 venues of each neighborhood-

In [21]:
df = df.rename(columns = {"STREET":"Neighborhood"}) 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

boston_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

boston_merged = boston_merged.dropna()
boston_merged = boston_merged.astype({"Cluster Labels": int}) 

boston_merged.head() # check the last columns!

,LON,LAT,Neighborhood,CITY,DISTRICT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,-71.494588,42.017492,CALIFORNIA AVENUE,BELLINGHAM,NORFOLK,2019,1,Liquor Store,American Restaurant,Pizza Place,Donut Shop,Chinese Restaurant,Convenience Store,Bar,Thai Restaurant,Pharmacy,Sandwich Place
3,-71.288031,42.017653,CEDAR STREET,FOXBOROUGH,NORFOLK,2035,0,Spa,Pharmacy,Café,Liquor Store,Sandwich Place,Chinese Restaurant,Park,Transportation Service,Construction & Landscaping,Grocery Store
5,-71.177637,42.057936,MANSFIELD STREET,SHARON,NORFOLK,2067,2,Construction & Landscaping,Exhibit,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop
6,-71.137240,42.074167,CLEARY DRIVE,STOUGHTON,NORFOLK,2072,1,Gym,Women's Store,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop
8,-71.280177,42.089123,COBBLE KNOLL DRIVE,WALPOLE,NORFOLK,2071,1,Gym,Gym / Fitness Center,Exhibit,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space


#### -Visualizing resulting clusters-

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['LAT'], boston_merged['LON'], boston_merged['Neighborhood'], boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

----

## Section 4 Examine Clusters

#### Cluster 1

#### key words: Restaurants, Grocery stores, Transportation Service, Entertainment Service, Gyms

In [32]:
boston_merged.loc[boston_merged['Cluster Labels'] == 0, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,LAT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,42.017653,2035,0,Spa,Pharmacy,Café,Liquor Store,Sandwich Place,Chinese Restaurant,Park,Transportation Service,Construction & Landscaping,Grocery Store
25,42.208523,2025,0,Café,Baseball Field,Fabric Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Women's Store
28,42.234212,2191,0,Park,Exhibit,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Women's Store
33,42.289019,2494,0,Park,Sandwich Place,Supermarket,Optical Shop,Gym / Fitness Center,Burrito Place,Donut Shop,Spa,Ice Cream Shop,Falafel Restaurant
35,42.406211,2150,0,Pizza Place,Yoga Studio,Fast Food Restaurant,Park,Donut Shop,Grocery Store,Convenience Store,Food,Intersection,Fish Market
45,42.289181,2459,0,Pizza Place,Fabric Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Women's Store
59,42.355245,2139,0,Park,Pizza Place,Coffee Shop,Track,Pool,Bike Rental / Bike Share,Electronics Store,Mattress Store,Gas Station,Candy Store
61,42.359890,2472,0,Pizza Place,Yoga Studio,Thai Restaurant,Coffee Shop,Park,Restaurant,Sandwich Place,Café,Electronics Store,Bridal Shop
73,42.390862,2149,0,Park,Building,Food & Drink Shop,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space
76,42.402243,2474,0,Food Truck,Intersection,Hotel,Park,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant


---

#### Cluster 2

#### key words: Restaurants, Concert Hall, Exhibit, Gym, Park

In [25]:
boston_merged.loc[boston_merged['Cluster Labels'] == 1, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,LAT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,42.017492,2019,1,Liquor Store,American Restaurant,Pizza Place,Donut Shop,Chinese Restaurant,Convenience Store,Bar,Thai Restaurant,Pharmacy,Sandwich Place
6,42.074167,2072,1,Gym,Women's Store,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop
8,42.089123,2071,1,Gym,Gym / Fitness Center,Exhibit,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space
9,42.101279,2081,1,Concert Hall,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
10,42.126208,2343,1,Pharmacy,Liquor Store,Baseball Field,Sandwich Place,Pizza Place,Chinese Restaurant,Gas Station,Bank,Ethiopian Restaurant,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,42.360117,2201,1,Italian Restaurant,Sandwich Place,Historic Site,Bakery,Coffee Shop,American Restaurant,Park,Pub,Seafood Restaurant,Donut Shop
181,42.348167,02199,1,Hotel,Clothing Store,Seafood Restaurant,Italian Restaurant,Coffee Shop,Plaza,Ice Cream Shop,Women's Store,American Restaurant,Bar
182,42.367341,02163,1,Park,Gym,College Stadium,Pool,College Hockey Rink,Residential Building (Apartment / Condo),Soccer Stadium,Squash Court,Tennis Court,Athletics & Sports
184,42.383251,2145,1,Mexican Restaurant,Bus Stop,Donut Shop,Liquor Store,Dog Run,Coffee Shop,Asian Restaurant,Arts & Entertainment,Metro Station,Bus Station


----

#### Cluster 3

#### key words: Construction & Landscaping, Beach, Stores

In [26]:
boston_merged.loc[boston_merged['Cluster Labels'] == 2, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,LAT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,42.057936,2067,2,Construction & Landscaping,Exhibit,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop
22,42.188598,2189,2,Construction & Landscaping,Clothing Store,Brewery,Optical Shop,Women's Store,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space
99,42.544018,1862,2,Construction & Landscaping,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
105,42.567261,1876,2,Beach,Construction & Landscaping,Lake,Women's Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
186,42.390782,2151,2,Beach,Construction & Landscaping,Pool,Women's Store,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit


---

#### Cluster 4

#### key words: Skating Rink, Women's Store, Exhibit

In [27]:
boston_merged.loc[boston_merged['Cluster Labels'] == 3, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,LAT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,42.134071,2054,3,Skating Rink,Women's Store,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop
51,42.314319,2493,3,Skating Rink,Women's Store,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop


---

#### Cluster 5

#### key words: Flower Shop, Exhibit, Restaurants

In [28]:
boston_merged.loc[boston_merged['Cluster Labels'] == 4, boston_merged.columns[[1] + list(range(5, boston_merged.shape[1]))]]

,LAT,POSTCODE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,42.189396,2090,4,Beach,Flower Shop,Falafel Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
65,42.368337,1749,4,Flower Shop,Women's Store,Fabric Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit
111,42.630206,1463,4,Flower Shop,Hot Dog Joint,Women's Store,Exhibit,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space


---